In [42]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis #esto es LDA
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split #generar set de prueba
from sklearn.linear_model import LogisticRegression #regresión logística
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt

Pre-procesamiento

In [43]:
car_data1=pd.read_excel('datos_cuentas1.xlsx')

In [44]:

car_data1['riesgo'] = pd.to_numeric(car_data1['riesgo'], errors='coerce')
car_data1["riesgo"]=car_data1["riesgo"].fillna(round(car_data1["riesgo"].mean(),1))

In [45]:

car_data1['riesgo'] = car_data1['riesgo'].apply(lambda x: 'no hay riesgo' if x < 1 else 'hay riesgo')
car_data1['riesgo']
df=car_data1['riesgo']
df


0        no hay riesgo
1        no hay riesgo
2        no hay riesgo
3        no hay riesgo
4        no hay riesgo
             ...      
22730       hay riesgo
22731       hay riesgo
22732       hay riesgo
22733    no hay riesgo
22734       hay riesgo
Name: riesgo, Length: 22735, dtype: object

In [46]:
car_data1['cp_cliente'] = pd.to_numeric(car_data1['cp_cliente'], errors='coerce',downcast='integer')
car_data1['cp_cliente']=car_data1['cp_cliente'].fillna('0')
car_data1['cp_cliente']=car_data1['cp_cliente'].astype('int64')

In [47]:

car_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   folio               22735 non-null  int64         
 1   tag                 22735 non-null  object        
 2   folio_solicitud     22735 non-null  int64         
 3   fecha               22735 non-null  datetime64[ns]
 4   marca               22735 non-null  object        
 5   modelo              22735 non-null  object        
 6   plazo               22735 non-null  object        
 7   precio              22735 non-null  float64       
 8   enganche            22735 non-null  float64       
 9   descuento           22735 non-null  float64       
 10  semana              22735 non-null  int64         
 11  monto_financiado    22735 non-null  float64       
 12  costo_total         22735 non-null  int64         
 13  monto_accesorios    22735 non-null  float64   

In [48]:
car_data2 = car_data1.select_dtypes(include=['int64', 'float64'])
car_data=pd.concat([df,car_data2],axis=1)

In [49]:
car_data

,riesgo,folio,folio_solicitud,precio,enganche,descuento,semana,monto_financiado,costo_total,monto_accesorios,...,inversion,pagos_realizados,reautorizacion,puntos,score_buro,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente
0,no hay riesgo,3,30,1949.0,780.0,0.0,81,1169.0,2106,0.0,...,0,0,0,0,0,40.02,5000,26,0,23.0
1,no hay riesgo,4,33,2999.0,1050.0,0.0,225,1949.0,2925,0.0,...,0,0,0,0,0,35.01,6000,13,0,60.0
2,no hay riesgo,5,37,1959.0,490.0,0.0,170,1469.0,2210,0.0,...,0,0,0,0,0,25.01,0,74,0,58.0
3,no hay riesgo,6,38,1959.0,490.0,0.0,170,1469.0,2210,0.0,...,0,0,0,0,0,25.01,7000,13,0,42.0
4,no hay riesgo,7,44,4299.0,1075.0,0.0,178,3224.0,6942,0.0,...,0,0,0,0,0,25.01,7000,74,0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,hay riesgo,22967,96502,3349.0,297.0,0.0,211,3052.0,5486,0.0,...,0,0,0,10,469,8.87,10000,0,97880,22.0
22731,hay riesgo,22968,96479,4100.0,1209.0,0.0,200,2891.0,5200,0.0,...,0,0,0,9,502,29.49,5000,0,37295,27.0
22732,hay riesgo,22969,96442,4849.0,1152.0,0.0,256,3697.0,6656,0.0,...,0,0,0,11,459,23.76,4750,0,20210,36.0
22733,no hay riesgo,22970,96536,9799.0,2799.0,0.0,386,7000.0,15054,0.0,...,0,0,0,14,699,28.56,7000,0,78434,53.0


In [50]:
car_data.isnull().sum()

riesgo              0
folio               0
folio_solicitud     0
precio              0
enganche            0
descuento           0
semana              0
monto_financiado    0
costo_total         0
monto_accesorios    0
status              0
fraude              0
inversion           0
pagos_realizados    0
reautorizacion      0
puntos              0
score_buro          0
porc_eng            0
limite_credito      0
semana_actual       0
cp_cliente          0
edad_cliente        9
dtype: int64

In [51]:
car_data['edad_cliente']=car_data['edad_cliente'].fillna('0')

In [52]:
car_data.isnull().sum()

riesgo              0
folio               0
folio_solicitud     0
precio              0
enganche            0
descuento           0
semana              0
monto_financiado    0
costo_total         0
monto_accesorios    0
status              0
fraude              0
inversion           0
pagos_realizados    0
reautorizacion      0
puntos              0
score_buro          0
porc_eng            0
limite_credito      0
semana_actual       0
cp_cliente          0
edad_cliente        0
dtype: int64

In [53]:
features = ['precio','enganche','monto_financiado','costo_total','inversion','score_buro','porc_eng','limite_credito','edad_cliente']
features1=['score_buro','edad_cliente','cp_cliente']
features2=['precio','enganche','monto_financiado','costo_total','monto_accesorios','inversion','score_buro','porc_eng','limite_credito','edad_cliente']
features3=['precio','enganche','monto_financiado','costo_total','inversion','score_buro','porc_eng','limite_credito','edad_cliente', 'cp_cliente']

In [54]:


# Seleccionamos las covariables y variables
X = car_data[features]
y = car_data['riesgo']
print(X.head)


# Pasado el procesamiento, partimos los datos en sets de prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamos el modelo de regresión logística.
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Realizamos predicciones con el set de prueba.
y_pred = logistic_model.predict(X_test)

# Evaluamos el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(confusion)

class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)



<bound method NDFrame.head of        precio  enganche  monto_financiado  costo_total  inversion  score_buro  \
0      1949.0     780.0            1169.0         2106          0           0   
1      2999.0    1050.0            1949.0         2925          0           0   
2      1959.0     490.0            1469.0         2210          0           0   
3      1959.0     490.0            1469.0         2210          0           0   
4      4299.0    1075.0            3224.0         6942          0           0   
...       ...       ...               ...          ...        ...         ...   
22730  3349.0     297.0            3052.0         5486          0         469   
22731  4100.0    1209.0            2891.0         5200          0         502   
22732  4849.0    1152.0            3697.0         6656          0         459   
22733  9799.0    2799.0            7000.0        15054          0         699   
22734  5399.0    1086.0            4313.0         7774          1           0  

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.86
Confusion Matrix:
[[2075  277]
 [ 365 1830]]
Classification Report:
               precision    recall  f1-score   support

   hay riesgo       0.85      0.88      0.87      2352
no hay riesgo       0.87      0.83      0.85      2195

     accuracy                           0.86      4547
    macro avg       0.86      0.86      0.86      4547
 weighted avg       0.86      0.86      0.86      4547



In [55]:


# Seleccionamos las covariables y variables
X = car_data[features1]
y = car_data['riesgo']
print(X.head)


# Pasado el procesamiento, partimos los datos en sets de prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamos el modelo de regresión logística.
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Realizamos predicciones con el set de prueba.
y_pred = logistic_model.predict(X_test)

# Evaluamos el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(confusion)

class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

<bound method NDFrame.head of        score_buro edad_cliente  cp_cliente
0               0         23.0           0
1               0         60.0           0
2               0         58.0           0
3               0         42.0           0
4               0         33.0           0
...           ...          ...         ...
22730         469         22.0       97880
22731         502         27.0       37295
22732         459         36.0       20210
22733         699         53.0       78434
22734           0         19.0       43800

[22735 rows x 3 columns]>
Accuracy: 0.76
Confusion Matrix:
[[1778  574]
 [ 519 1676]]
Classification Report:
               precision    recall  f1-score   support

   hay riesgo       0.77      0.76      0.76      2352
no hay riesgo       0.74      0.76      0.75      2195

     accuracy                           0.76      4547
    macro avg       0.76      0.76      0.76      4547
 weighted avg       0.76      0.76      0.76      4547



In [56]:
# Seleccionamos las covariables y variables
X = car_data[features2]
y = car_data['riesgo']
print(X.head)


# Pasado el procesamiento, partimos los datos en sets de prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamos el modelo de regresión logística.
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Realizamos predicciones con el set de prueba.
y_pred = logistic_model.predict(X_test)

# Evaluamos el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(confusion)

class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)



<bound method NDFrame.head of        precio  enganche  monto_financiado  costo_total  monto_accesorios  \
0      1949.0     780.0            1169.0         2106               0.0   
1      2999.0    1050.0            1949.0         2925               0.0   
2      1959.0     490.0            1469.0         2210               0.0   
3      1959.0     490.0            1469.0         2210               0.0   
4      4299.0    1075.0            3224.0         6942               0.0   
...       ...       ...               ...          ...               ...   
22730  3349.0     297.0            3052.0         5486               0.0   
22731  4100.0    1209.0            2891.0         5200               0.0   
22732  4849.0    1152.0            3697.0         6656               0.0   
22733  9799.0    2799.0            7000.0        15054               0.0   
22734  5399.0    1086.0            4313.0         7774               0.0   

       inversion  score_buro  porc_eng  limite_credito ed

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [57]:
# Seleccionamos las covariables y variables
X = car_data[features3]
y = car_data['riesgo']
print(X.head)


# Pasado el procesamiento, partimos los datos en sets de prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamos el modelo de regresión logística.
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Realizamos predicciones con el set de prueba.
y_pred = logistic_model.predict(X_test)

# Evaluamos el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(confusion)

class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)



<bound method NDFrame.head of        precio  enganche  monto_financiado  costo_total  inversion  score_buro  \
0      1949.0     780.0            1169.0         2106          0           0   
1      2999.0    1050.0            1949.0         2925          0           0   
2      1959.0     490.0            1469.0         2210          0           0   
3      1959.0     490.0            1469.0         2210          0           0   
4      4299.0    1075.0            3224.0         6942          0           0   
...       ...       ...               ...          ...        ...         ...   
22730  3349.0     297.0            3052.0         5486          0         469   
22731  4100.0    1209.0            2891.0         5200          0         502   
22732  4849.0    1152.0            3697.0         6656          0         459   
22733  9799.0    2799.0            7000.0        15054          0         699   
22734  5399.0    1086.0            4313.0         7774          1           0  

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
